In [ ]:
# This code performs the inference for all the fine-tuned models
# Runs of CPU/GPU, faster on GPU

# Similar to baselines.ipynb but the models are different
# Please upload CSFCube-master.zip to the colab notebook
# This zip file is present in our code base
# fine-tuned models loaded from Google Drive

# **Install libraries and prepare CSF-Cube dataset**

In [ ]:
!pip install transformers
!pip install gdown
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=e6a4f956e2aec02d2b67032e9c83e7085c358f1a2f17e6cb82ec8a06eea71a4c
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527

In [ ]:
!unzip CSFCube-master.zip

Archive:  CSFCube-master.zip
   creating: CSFCube-master/
  inflating: CSFCube-master/.gitignore  
  inflating: CSFCube-master/LICENSE.md  
  inflating: CSFCube-master/README.md  
  inflating: CSFCube-master/abstracts-csfcube-preds.jsonl  
  inflating: CSFCube-master/ann_guidelines.pdf  
  inflating: CSFCube-master/datasheet.md  
   creating: CSFCube-master/eval_scripts/
  inflating: CSFCube-master/eval_scripts/rank_metrics.py  
  inflating: CSFCube-master/eval_scripts/ranking_eval.py  
  inflating: CSFCube-master/eval_scripts/requirements.txt  
   creating: CSFCube-master/eval_scripts/sample_ranked_pools/
  inflating: CSFCube-master/eval_scripts/sample_ranked_pools/test-pid2pool-csfcube-specter-background-ranked.json  
  inflating: CSFCube-master/eval_scripts/sample_ranked_pools/test-pid2pool-csfcube-specter-method-ranked.json  
  inflating: CSFCube-master/eval_scripts/sample_ranked_pools/test-pid2pool-csfcube-specter-result-ranked.json  
  inflating: CSFCube-master/evaluation_splits.

In [ ]:
!gdown https://drive.google.com/uc?id=1THQvaypgyLgpOSHdHRMca7j1Rb44yLVe

Downloading...
From: https://drive.google.com/uc?id=1THQvaypgyLgpOSHdHRMca7j1Rb44yLVe
To: /content/specter_triplet_4.pt
100% 440M/440M [00:06<00:00, 63.7MB/s]


In [ ]:
!gdown https://drive.google.com/uc?id=1PG7_SfihyL1OHEPR32SbFISqY6wyu4xc

Downloading...
From: https://drive.google.com/uc?id=1PG7_SfihyL1OHEPR32SbFISqY6wyu4xc
To: /content/scincl_triplet_4.pt
100% 440M/440M [00:04<00:00, 101MB/s] 


In [ ]:
!gdown https://drive.google.com/uc?id=1ICTJSb6uziPbmbI4J12RnONNOx07y_O8

Downloading...
From: https://drive.google.com/uc?id=1ICTJSb6uziPbmbI4J12RnONNOx07y_O8
To: /content/finetuned-sentbert-pp-parade-train.pt
100% 268M/268M [00:01<00:00, 217MB/s]


# **Parse CSF-Cube dataset into Python dictionaries**

In [ ]:
import json


def parse_dataset(file_data, file_background, file_method, file_result):
    data = dict()
    with open(file_data, 'r') as f:
        abs_list = list(f)
    for paper in abs_list:
        paper = json.loads(paper)
        dict_paper = dict()
        dict_paper['title'] = paper['title']
        dict_paper['abstract'] = paper['abstract']
        dict_paper['pred_labels'] = paper['pred_labels']
        try:
            print(data[paper['paper_id']])
        except:
            data[paper['paper_id']] = dict_paper
    file_list = [file_background, file_method, file_result]
    result_list = []
    for f in file_list:
        with open(f, 'r') as json_file:
            results_data = json.load(json_file)
        result_dict = dict()
        for k in results_data.keys():
            try:
                print(result_dict[k].keys())
            except:
                result_dict[k] = dict()
            result_dict[k]['cands'] = results_data[k]['cands']
            result_dict[k]['relevance_adju'] = results_data[k]['relevance_adju']
        result_list.append(result_dict)
    return data, result_list[0], result_list[1], result_list[2]

In [ ]:
sentences,bg_ranking,method_ranking,result_ranking = parse_dataset('/content/CSFCube-master/abstracts-csfcube-preds.jsonl','/content/CSFCube-master/test-pid2anns-csfcube-background.json','/content/CSFCube-master/test-pid2anns-csfcube-method.json','/content/CSFCube-master/test-pid2anns-csfcube-result.json')

# **Load models**

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch
from sentence_transformers import SentenceTransformer

def init_model_tokenizer(i):
    if(i==0):
        tokenizer = AutoTokenizer.from_pretrained('allenai/specter')
        model = AutoModel.from_pretrained('allenai/specter')
    if i==3:
        tokenizer = AutoTokenizer.from_pretrained('malteos/scincl')
        model = AutoModel.from_pretrained('malteos/scincl')
    if i==10:
        model = SentenceTransformer('paraphrase-TinyBERT-L6-v2')
        tokenizer = None

    return model,tokenizer

In [ ]:
def baseline_model(model_id, model, tokenizer, texts, max_length):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()
    if model_id == 10:
        model.eval()
        cls_embeddings = model.encode(texts)
        return cls_embeddings
    with torch.no_grad():
        tokens = tokenizer(texts, max_length=max_length, truncation=True, padding=True, return_tensors='pt')
        tokens.to(device)
        outputs = model(**tokens)

        # Get the output embeddings
        embeddings = outputs.last_hidden_state
        cls_embedding = embeddings[:, 0, :]
        cls_embedding = cls_embedding.cpu().detach().numpy()
        return cls_embedding

In [ ]:
keys = [0,3,10]

model_dict = dict()
tokenizer_dict = dict()

for k in keys:
    model_dict[k],tokenizer_dict[k] = init_model_tokenizer(k)
    model = model_dict[k]
    if k==0:
        model.load_state_dict(torch.load('/content/specter_triplet_4.pt', map_location=torch.device('cpu')))
    elif k==3:
        model.load_state_dict(torch.load('/content/scincl_triplet_4.pt', map_location=torch.device('cpu')))
    else:
        model.load_state_dict(torch.load('/content/finetuned-sentbert-pp-parade-train.pt', map_location=torch.device('cpu')))
    model_dict[k] = model

# **Ranked retrieval for candidates**

In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def L2_dist_ranking(cls_embeddings,query_id,cand_ids,results_dict):
    query_cls = cls_embeddings[0]
    # print(query_cls.shape,cls_embeddings.shape,len(cand_ids))
    if query_id in results_dict.keys():
        print('err')
        return
    results = []
    for i in range(1,len(cls_embeddings)):
        l2_norm = np.linalg.norm(query_cls - cls_embeddings[i])
        results.append([cand_ids[i-1],l2_norm])
    # print(results)
    results = sorted(results, key = lambda x: x[1])
    
    for i in range(len(results)):
        results[i][1] = float(results[i][1])
    # print(results)
    results_dict[query_id] = results
    return

def cosine_similarity(query_embeddings, cls_embeddings):
    max_cosine_sim = 0.0
    for embd_query in query_embeddings:
        for embd_cand in cls_embeddings:
            curr_cosine_sim = dot(embd_query, embd_cand)/(norm(embd_cand)*norm(embd_query))
            if curr_cosine_sim > max_cosine_sim:
                max_cosine_sim = curr_cosine_sim

    return max_cosine_sim

In [ ]:
import json

def write_results(results_dict,model_name,facet):
    file_name = "test-pid2pool-csfcube-" + model_name + "-" + facet + "-ranked.json"
    with open(file_name,'w') as f:
        json.dump(results_dict,f)


In [ ]:
import json
import os

def prepare_batch(query_id,data_dict,ranking_dict,query_facet,cand_facet):
    titles = []
    abstracts = []
    query_title = data_dict[query_id]['title']
    titles.append(query_title)
    query_abstract = []
    for i in range(len(data_dict[query_id]['abstract'])):
        curr_label = data_dict[query_id]['pred_labels'][i]
        if query_facet == "abstract" or query_facet == curr_label:
            query_abstract.append(data_dict[query_id]['abstract'][i])
        elif query_facet == "background_label" and curr_label=="objective_label":
            query_abstract.append(data_dict[query_id]['abstract'][i])
        elif query_facet == "objective_label" and curr_label=="background_label":
            query_abstract.append(data_dict[query_id]['abstract'][i])
    abstracts.append(query_abstract)

    for j in range(len(ranking_dict[query_id]['cands'])):
        cand_id = ranking_dict[query_id]['cands'][j]
        cand_title = data_dict[cand_id]['title']
        titles.append(cand_title)
        cand_abstract = []
        for i in range(len(data_dict[cand_id]['abstract'])):
            curr_label = data_dict[cand_id]['pred_labels'][i]
            if cand_facet == "abstract" or cand_facet == curr_label:
                cand_abstract.append(data_dict[cand_id]['abstract'][i])
            elif cand_facet == "background_label" and curr_label=="objective_label":
                cand_abstract.append(data_dict[cand_id]['abstract'][i])
            elif cand_facet == "objective_label" and curr_label=="background_label":
                cand_abstract.append(data_dict[cand_id]['abstract'][i])
        abstracts.append(cand_abstract)    

    return titles,abstracts

def eval_model(model_id, model_name, data_dict, ranking_dict, query_facet, cand_facet, ranking_facet):
    model, tokenizer = model_dict[model_id], tokenizer_dict[model_id]
    results_dict = dict()
    for query_id in ranking_dict.keys():
        batch_data = prepare_batch(query_id, data_dict, ranking_dict, query_facet, cand_facet)
        if model_name == 'specter' or model_name == 'scincl' or model_name == 'scibert_uncased':
            for i in range(len(batch_data[1])):
                abs = ""
                for j in batch_data[1][i]:
                    abs += j
                batch_data[1][i] = abs
            title_abs = [batch_data[0][i] + tokenizer.sep_token +batch_data[1][i] for i in range(len(batch_data[0]))]
            cls_embeddings = baseline_model(model_id, model, tokenizer, title_abs, 512)
            L2_dist_ranking(cls_embeddings, query_id,ranking_dict[query_id]['cands'], results_dict)
        else:
            abstracts = batch_data[1]
            results = []
            if (len(batch_data[1][0]) == 0):
                batch_data[1][0] = ['']
            query_embeddings = baseline_model(10, model, None, batch_data[1][0], None)
            for i in range(1, len(batch_data[1])):
                if len(batch_data[1][i]) == 0:
                    batch_data[1][i] = ['']
                cls_embeddings = baseline_model(10, model, None, batch_data[1][i], None)
                max_cosine_sim = cosine_similarity(query_embeddings, cls_embeddings)
                results.append([ranking_dict[query_id]['cands'][i-1], max_cosine_sim])
            results = sorted(results, key=lambda x: x[1], reverse=True)
            for i in range(len(results)):
                results[i][1] = float(results[i][1])
            results_dict[query_id] = results
        write_results(results_dict, model_name, ranking_facet)


# specter

### Query Abstract - Candidate Abstract

In [ ]:
eval_model(0,'specter',sentences,bg_ranking,'abstract','abstract','background')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment specter --facet background
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2324, 0.2563, 0.4114, 0.7706, 0.5686, 0.5590

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1929, 0.2812, 0.4548, 0.7862, 0.5823, 0.5874

/content


In [ ]:
eval_model(0,'specter',sentences,method_ranking,'abstract','abstract','method')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment specter --facet method
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1235, 0.1500, 0.5350, 0.6403, 0.4127, 0.4099

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1153, 0.1389, 0.4781, 0.6439, 0.4159, 0.4100

/content


In [ ]:
eval_model(0,'specter',sentences,result_ranking,'abstract','abstract','result')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment specter --facet result
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1372, 0.2375, 0.5394, 0.7655, 0.5949, 0.5893

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1350, 0.2382, 0.5432, 0.7259, 0.5344, 0.5353

/content


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment specter --facet all
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1644, 0.2146, 0.4953, 0.7255, 0.5254, 0.5194

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1473, 0.2179, 0.4919, 0.7169, 0.5090, 0.5087

/cont

### Query facet - Candidate Abstract

In [ ]:
eval_model(0,'specter',sentences,bg_ranking,'background_label','abstract','background')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment specter --facet background
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2239, 0.3000, 0.4427, 0.7596, 0.5747, 0.5687

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1904, 0.2969, 0.4660, 0.7717, 0.5807, 0.5852

/content


In [ ]:
eval_model(0,'specter',sentences,method_ranking,'method_label','abstract','method')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment specter --facet method
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1976, 0.1188, 0.4115, 0.6132, 0.3669, 0.3754

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1539, 0.1149, 0.3906, 0.6146, 0.3603, 0.3625

/content


In [ ]:
eval_model(0,'specter',sentences,result_ranking,'result_label','abstract','result')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment specter --facet result
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1384, 0.2000, 0.4685, 0.7397, 0.5390, 0.5478

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1429, 0.2167, 0.4599, 0.7303, 0.5247, 0.5410

/content


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment specter --facet all
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1866, 0.2063, 0.4409, 0.7042, 0.4936, 0.4973

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1620, 0.2079, 0.4378, 0.7040, 0.4866, 0.4942

/cont

### Query facet - Candidate facet

In [ ]:
eval_model(0,'specter',sentences,bg_ranking,'background_label','background_label','background')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment specter --facet background
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2019, 0.2687, 0.4001, 0.7833, 0.6101, 0.6062

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1786, 0.2625, 0.4054, 0.7793, 0.5857, 0.5928

/content


In [ ]:
eval_model(0,'specter',sentences,method_ranking,'method_label','method_label','method')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment specter --facet method
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2014, 0.1125, 0.3802, 0.6462, 0.3904, 0.3908

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1559, 0.1035, 0.3521, 0.6245, 0.3626, 0.3704

/content


In [ ]:
eval_model(0,'specter',sentences,result_ranking,'result_label','result_label','result')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment specter --facet result
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1455, 0.2187, 0.5213, 0.7349, 0.5423, 0.5369

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1443, 0.2344, 0.5038, 0.7261, 0.5251, 0.5328

/content


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment specter --facet all
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1829, 0.2000, 0.4339, 0.7215, 0.5142, 0.5113

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-specter-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1592, 0.1990, 0.4204, 0.7085, 0.4893, 0.4969

/cont

# scincl

### Query Abstract - Candidate Abstract

In [ ]:
eval_model(3,'scincl',sentences,bg_ranking,'abstract','abstract','background')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scincl --facet background
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2745, 0.3250, 0.4915, 0.8038, 0.6298, 0.6261

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2370, 0.3031, 0.4760, 0.8143, 0.6379, 0.6445

/content


In [ ]:
eval_model(3,'scincl',sentences,method_ranking,'abstract','abstract','method')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scincl --facet method
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1000, 0.1500, 0.4829, 0.6703, 0.4434, 0.4381

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1013, 0.1472, 0.4326, 0.6823, 0.4616, 0.4671

/content


In [ ]:
eval_model(3,'scincl',sentences,result_ranking,'abstract','abstract','result')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scincl --facet result
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1831, 0.2563, 0.5765, 0.8020, 0.6417, 0.6366

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1965, 0.2559, 0.5262, 0.7717, 0.5886, 0.5948

/content


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scincl --facet all
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1859, 0.2437, 0.5170, 0.7587, 0.5716, 0.5669

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1777, 0.2344, 0.4779, 0.7548, 0.5609, 0.5671

/conten

### Query facet - Candidate Abstract

In [ ]:
eval_model(3,'scincl',sentences,bg_ranking,'background_label','abstract','background')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scincl --facet background
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2827, 0.3563, 0.5570, 0.8167, 0.6762, 0.6763

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2311, 0.2969, 0.4582, 0.8072, 0.6276, 0.6363

/content


In [ ]:
eval_model(3,'scincl',sentences,method_ranking,'method_label','abstract','method')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scincl --facet method
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0985, 0.1438, 0.4754, 0.6584, 0.4275, 0.4180

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1005, 0.1385, 0.4365, 0.6604, 0.4277, 0.4305

/content


In [ ]:
eval_model(3,'scincl',sentences,result_ranking,'result_label','abstract','result')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scincl --facet result
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1678, 0.2063, 0.4674, 0.7271, 0.5138, 0.5161

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1757, 0.2365, 0.4868, 0.7294, 0.5242, 0.5343

/content


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scincl --facet all
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1830, 0.2354, 0.4999, 0.7341, 0.5392, 0.5368

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1686, 0.2235, 0.4617, 0.7310, 0.5253, 0.5324

/conten

### Query facet - Candidate facet

In [ ]:
eval_model(3,'scincl',sentences,bg_ranking,'background_label','background_label','background')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scincl --facet background
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2868, 0.3625, 0.5538, 0.7995, 0.6363, 0.6353

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2188, 0.3375, 0.5197, 0.8029, 0.6253, 0.6334

/content


In [ ]:
eval_model(3,'scincl',sentences,method_ranking,'method_label','method_label','method')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scincl --facet method
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1030, 0.1375, 0.4427, 0.6413, 0.4132, 0.4149

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0996, 0.1243, 0.3893, 0.6580, 0.4179, 0.4174

/content


In [ ]:
eval_model(3,'scincl',sentences,result_ranking,'result_label','result_label','result')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scincl --facet result
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1752, 0.2188, 0.5023, 0.7291, 0.5344, 0.5350

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1618, 0.2177, 0.4610, 0.7280, 0.5213, 0.5339

/content


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment scincl --facet all
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1883, 0.2396, 0.4996, 0.7233, 0.5280, 0.5284

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-scincl-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1597, 0.2246, 0.4553, 0.7282, 0.5196, 0.5262

/conten

# sentbert-pp

### Query facet - Candidate Abstract

In [ ]:
eval_model(10, 'sentbert-pp', sentences, bg_ranking, 'background_label', 'abstract', 'background')

In [ ]:
!python3 /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path /content/CSFCube-master/ --ranked_path ./ --experiment sentbert-pp --facet background

EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ./test-pid2pool-csfcube-sentbert-pp-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2113, 0.3125, 0.4744, 0.7800, 0.6023, 0.5992

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ./test-pid2pool-csfcube-sentbert-pp-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2137, 0.3062, 0.4789, 0.7919, 0.6059, 0.6118



In [ ]:
eval_model(10, 'sentbert-pp', sentences, method_ranking, 'method_label', 'abstract', 'method')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-pp --facet method
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1008, 0.0938, 0.3884, 0.5833, 0.3204, 0.3208

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1084, 0.1052, 0.3671, 0.5979, 0.3313, 0.3330

/content


In [ ]:
eval_model(10, 'sentbert-pp', sentences, result_ranking, 'result_label', 'abstract', 'result')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-pp --facet result
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1321, 0.2250, 0.5178, 0.7648, 0.5777, 0.5800

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1298, 0.2042, 0.4706, 0.7176, 0.5014, 0.5144

/content


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-pp --facet all
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1481, 0.2104, 0.4602, 0.7094, 0.5001, 0.5000

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1494, 0.2033, 0.4376, 0.7004, 0.4766, 0.483

### Query Abstract - Candidate Abstract

In [ ]:
eval_model(10, 'sentbert-pp', sentences, bg_ranking, 'abstract', 'abstract', 'background')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-pp --facet background
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2081, 0.3125, 0.4411, 0.7655, 0.5756, 0.5810

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2009, 0.2969, 0.4458, 0.7743, 0.5770, 0.5853

/content


In [ ]:
eval_model(10, 'sentbert-pp', sentences, method_ranking, 'abstract', 'abstract', 'method')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-pp --facet method
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0983, 0.1313, 0.3557, 0.5817, 0.3254, 0.3182

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0979, 0.1267, 0.3799, 0.6049, 0.3554, 0.3573

/content


In [ ]:
eval_model(10, 'sentbert-pp', sentences, result_ranking, 'abstract', 'abstract', 'result')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-pp --facet result
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1635, 0.2125, 0.4632, 0.7580, 0.5771, 0.5794

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1480, 0.1868, 0.3942, 0.6987, 0.4791, 0.4927

/content


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-pp --facet all
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1566, 0.2188, 0.4200, 0.7017, 0.4927, 0.4928

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1479, 0.2017, 0.4055, 0.6907, 0.4680, 0.476

### Query facet - Candidate facet

In [ ]:
eval_model(10, 'sentbert-pp', sentences, bg_ranking, 'background_label', 'background_label', 'background')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-pp --facet background
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2204, 0.3000, 0.4579, 0.7936, 0.6175, 0.6189

EVAL SPLIT: test
Gold query pids: 16
Valid ranked query pids: 16
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-background-perq.csv
Total queries: 16; Total candidates: 1877
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.2251, 0.3156, 0.5010, 0.8033, 0.6242, 0.6316

/content


In [ ]:
eval_model(10, 'sentbert-pp', sentences, method_ranking, 'method_label', 'method_label', 'method')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-pp --facet method
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0827, 0.1063, 0.3646, 0.5849, 0.3340, 0.3396

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-method-perq.csv
Total queries: 17; Total candidates: 2174
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.0873, 0.1087, 0.3189, 0.5848, 0.3089, 0.3168

/content


In [ ]:
eval_model(10, 'sentbert-pp', sentences, result_ranking, 'result_label', 'result_label', 'result')

In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-pp --facet result
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1243, 0.2250, 0.5118, 0.7531, 0.5571, 0.5681

EVAL SPLIT: test
Gold query pids: 17
Valid ranked query pids: 17
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-result-perq.csv
Total queries: 17; Total candidates: 2193
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1104, 0.2125, 0.4746, 0.7188, 0.5068, 0.5230

/content


In [ ]:
%cd CSFCube-master
!python /content/CSFCube-master/eval_scripts/ranking_eval.py eval_pool_ranking --gold_path ./ --ranked_path ../ --experiment sentbert-pp --facet all
%cd ..

/content/CSFCube-master
EVAL SPLIT: dev
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1424, 0.2104, 0.4447, 0.7106, 0.5029, 0.5089

EVAL SPLIT: test
Reading facet: background
Gold query pids: 16
Valid ranked query pids: 16
Reading facet: method
Gold query pids: 17
Valid ranked query pids: 17
Reading facet: result
Gold query pids: 17
Valid ranked query pids: 17
Total queries: 50
Precision and recall at rank: 20
Wrote: ../test-pid2pool-csfcube-sentbert-pp-all-perq.csv
Total queries: 50; Total candidates: 6244
R-Precision; Precision@20; Recall@20; NDCG; NDCG@20; NDCG%20
0.1392, 0.2100, 0.4291, 0.7000, 0.4766, 0.487